# International Faculty Development Program on _“Quantum Artificial Intelligence”_

### Quantum $k$-means Clustering Algorithm

- $k$ means clustering is a unsupervised machine learning algorithm used for clustering.
- The objective of this algorithm is essentially a grouping of datapoints based on how similar and different they are to one another.
- This algorithm finds $k$ centers for $k$ different clusters and assigns data points to one of the K clusters depending on their distance from the center of the clusters.

![k_means](https://miro.medium.com/v2/resize:fit:720/format:webp/1*fz-rjYPPRlGEMdTI-RLbDg.png)

__Algorithm__

`Input:`
* Datapoints $\{\mathbf x_0,\mathbf x_1,\dots,\mathbf x_{n-1}\}$.
* Labels $\{y_0,y_1,\dots,y_{n-1}\}$.
* [optional] Number of cluster $k$.

`Output:` $k$ different clusters.

1. Initialize random centers for each cluster.
2. Assign each datapoints with their nearest center. This will generate initial $k$ clusters. (Here we will use swap test)
3. Calculate new centers for each cluster taking mean over datapoints assign to the cluster.
4. Repeat step 2 and 3 until convergence.

__This algorithm may not converge correctly if initial choice of center is bad.__

In [ ]:
def swap_test(qc, swap):
    '''Performs swap test and return probability of |0⟩
    Arguments:
        qc: A quantum circuit to perform swap test
        swap: Target qubits (to be swapped)
    '''
    from qiskit_aer import AerSimulator
    from qiskit import ClassicalRegister, QuantumRegister
    simulator = AerSimulator()
    
    meas = ClassicalRegister(1)
    targ = QuantumRegister(1)
    qc.add_register(meas, targ)
    qc.h(targ)
    qc.cswap(targ, swap[0], swap[1])
    qc.h(targ)
    qc.measure(targ, 0)
    
    shots = 1000
    
    counts = simulator.run(qc, shots = shots).result().get_counts()
    
    return counts['0']/shots

In [ ]:
def get_Distance(A, B, sim):
    '''Calculates distance between two vectors A and B using swap test'''
    from numpy import linalg, sqrt, log2
    from qiskit import QuantumRegister, QuantumCircuit

    norm_A = linalg.norm(A) 
    norm_B = linalg.norm(B)
    C = norm_A**2 + norm_B**2

    # Create phi and psi state with the data
    psi = [norm_A/sqrt(C), -norm_B/sqrt(C)]
    phi = [a/(norm_A*sqrt(2)) for a in A] + [b/(norm_B*sqrt(2)) for b in B]


    q1 = QuantumRegister(1, name = 'psi')
    q2 = QuantumRegister(int(log2(len(A)))+1, name = 'phi')
    qc = QuantumCircuit(q1,q2)

    # States initialization
    qc.initialize(psi, q1)
    qc.initialize(phi, q2)

    # Apply SWAP test
    return 2*C*(2*swap_test(qc, (q1, q2[-1]), sim) - 1)

In [ ]:
def find_nearest_neighbour(points, centers, sim):
    '''Finds nearest center for each data points and assign label accordingly
    Arguments:
        points: Data points
        center: Current centers of the clusters
    '''
    from numpy import zeros
    
    n = len(points)
    k = centers.shape[0]
    labels = zeros(n)
    
    # Loop over all data points
    for i in range(n):
        min_dist = 10000
        ind = 0
        
        # Find nearest center for data point
        # Later we will see that this can be done in one call using quantum parallelism
        for j in range(k):
            temp_dist = get_Distance(points[i,:], centers[j,:], sim)
            if temp_dist < min_dist:
                min_dist = temp_dist
                ind = j
                
        # Assign nearest label
        labels[i] = ind
    
    return labels

In [ ]:
def find_centers(points, labels):
    '''Finds new centers based on new labels
    Arguments:
        points: Data points
        labels: Data labels
    '''
    from numpy import zeros, max as mx, average
    
    n = len(points)
    k = int(mx(labels))+1
   
    centers = zeros([k,2])
    
    # Calculate new centers for clusters by taking average of all data points from corresponding cluster
    for i in range(k):
        centers[i,:] = average(points[labels==i], axis = 0)
        
    return centers

In [ ]:
from numpy import ndarray
from numpy.random import randint
from sklearn.datasets import make_blobs
from matplotlib.pyplot import subplots, scatter, show

# Create data points
data_size = 100
centers = [[0, 0], [2, 0], [2, 4]]
points, labels = make_blobs(n_samples = data_size, n_features = 2, centers = centers, cluster_std = 0.5)

# Initialize with random centers
centers = points[randint(points.shape[0], size = len(centers)),:]

# Visualize the data
_, ax = subplots(1, 1)
scatter(points[:,0], points[:,1])
scatter(centers[:,0], centers[:,1], c = 'red', marker = 's')
show()

In [ ]:
for _ in range(5):
    labels = find_nearest_neighbour(points, centers, True)
    # Find new centers
    new_centers = find_centers(points, labels)
    
    # Visualize the classification
    _, ax = subplots(1, 1)
    scatter(points[:,0], points[:,1], c = labels, cmap = 'viridis')
    scatter(centers[:,0], centers[:,1], c = 'red', marker = 's')
    scatter(new_centers[:,0], new_centers[:,1], c = 'green', marker = 's')
    show()
    
    # Assign new centers
    centers = new_centers

#### Reference

1. S. DiAdamo, C. O’Meara, G. Cortiana and J. Bernabé-Moreno, "Practical Quantum K-Means Clustering: Performance Analysis and Applications in Energy Grid Classification," in IEEE Transactions on Quantum Engineering, vol. 3, pp. 1-16, 2022.
2. https://www.geeksforgeeks.org/k-means-clustering-introduction/